# 📊 SerDes Eye Diagram Dashboard - Jupyter Notebook

This notebook demonstrates the **Jupyter-based SVF Eye Diagram Dashboard** capabilities, providing interactive visualization of:
- Captured waveforms
- Eye diagrams with automatic measurements
- Pass/fail annotations from SVF analyzers
- Multi-protocol analysis (USB4, PCIe, Ethernet)

## 🚀 Features
- **Interactive Controls**: Real-time parameter adjustment
- **Multi-Lane Analysis**: Support for multi-lane protocols
- **Professional Visualizations**: Publication-ready plots
- **SVF Integration**: Direct integration with framework analyzers
- **Export Capabilities**: Save results and plots

## 📋 Setup and Dependencies

In [ ]:
# Import required libraries
import sys
import numpy as np
from pathlib import Path

# Add framework to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

# Import SerDes Validation Framework
from serdes_validation_framework.jupyter_dashboard import (
    EyeDiagramDashboard, 
    create_dashboard,
    DashboardConfig,
    check_dashboard_dependencies,
    display_requirements
)

from serdes_validation_framework.jupyter_dashboard import (
    WaveformAnalyzer,
    InteractivePlotter,
    check_interactive_dependencies
)

# Check dependencies
print("🔍 Checking Dashboard Dependencies:")
deps = check_dashboard_dependencies()
print("\n🎨 Checking Interactive Plot Dependencies:")
interactive_deps = check_interactive_dependencies()

## 📊 Generate Sample Data

Let's create realistic SerDes signal data for different protocols:

In [ ]:
def generate_usb4_signal(duration=1e-6, sample_rate=40e9, num_lanes=2):
    """Generate USB4 dual-lane signal data"""
    t = np.linspace(0, duration, int(duration * sample_rate))
    
    signal_data = {}
    for lane in range(num_lanes):
        # Generate NRZ-like signal with some realistic characteristics
        data_bits = np.random.choice([-1, 1], size=len(t))
        
        # Add some filtering (bandwidth limitation)
        from scipy import signal as scipy_signal
        b, a = scipy_signal.butter(4, 0.3)  # Low-pass filter
        filtered_signal = scipy_signal.filtfilt(b, a, data_bits)
        
        # Scale to USB4 voltage levels (±400mV)
        usb4_signal = filtered_signal * 0.4
        
        # Add some noise and jitter
        noise = np.random.normal(0, 0.02, len(usb4_signal))
        jitter = np.random.normal(0, 0.01, len(usb4_signal))
        
        # Apply lane-specific offset
        lane_offset = lane * 0.05  # Small offset between lanes
        
        signal_data[f'lane_{lane}'] = usb4_signal + noise + jitter + lane_offset
    
    return signal_data, t

def generate_pcie_signal(duration=1e-6, sample_rate=80e9):
    """Generate PCIe PAM4 signal data"""
    t = np.linspace(0, duration, int(duration * sample_rate))
    
    # Generate PAM4 signal (4 levels: -3, -1, +1, +3)
    pam4_levels = np.random.choice([-3, -1, 1, 3], size=len(t))
    
    # Add some filtering
    from scipy import signal as scipy_signal
    b, a = scipy_signal.butter(4, 0.2)
    filtered_signal = scipy_signal.filtfilt(b, a, pam4_levels)
    
    # Scale to PCIe voltage levels (±600mV)
    pcie_signal = filtered_signal * 0.2
    
    # Add noise
    noise = np.random.normal(0, 0.03, len(pcie_signal))
    
    return {'lane_0': pcie_signal + noise}, t

def generate_ethernet_signal(duration=1e-6, sample_rate=224e9):
    """Generate Ethernet 224G PAM4 signal data"""
    t = np.linspace(0, duration, int(duration * sample_rate))
    
    # Generate PAM4 signal for high-speed Ethernet
    pam4_levels = np.random.choice([-3, -1, 1, 3], size=len(t))
    
    # Add filtering for high-speed characteristics
    from scipy import signal as scipy_signal
    b, a = scipy_signal.butter(6, 0.15)
    filtered_signal = scipy_signal.filtfilt(b, a, pam4_levels)
    
    # Scale to Ethernet voltage levels (±400mV)
    eth_signal = filtered_signal * 0.15
    
    # Add noise (higher for high-speed)
    noise = np.random.normal(0, 0.025, len(eth_signal))
    
    return {'lane_0': eth_signal + noise}, t

# Generate sample data for all protocols
print("🔄 Generating sample signal data...")

# USB4 data
usb4_signals, usb4_time = generate_usb4_signal(duration=2e-6, num_lanes=2)
print(f"✅ USB4: {len(usb4_signals)} lanes, {len(usb4_time)} samples")

# PCIe data  
pcie_signals, pcie_time = generate_pcie_signal(duration=1e-6)
print(f"✅ PCIe: {len(pcie_signals)} lanes, {len(pcie_time)} samples")

# Ethernet data
eth_signals, eth_time = generate_ethernet_signal(duration=0.5e-6)
print(f"✅ Ethernet: {len(eth_signals)} lanes, {len(eth_time)} samples")

print("\n🎯 Sample data generation complete!")

## 🎨 Basic Eye Diagram Dashboard

Let's start with a basic eye diagram dashboard for USB4:

In [ ]:
# Create USB4 dashboard
print("🔄 Creating USB4 Eye Diagram Dashboard...")

# Create dashboard with custom configuration
config = DashboardConfig(
    figure_width=12,
    figure_height=8,
    show_measurements=True,
    show_mask=True,
    background_color='white'
)

usb4_dashboard = create_dashboard(
    signal_data=usb4_signals,
    sample_rate=40e9,
    protocol="USB4",
    config=config
)

print("✅ USB4 dashboard created!")

In [ ]:
# Analyze eye diagram for lane 0
print("🔍 Analyzing USB4 Lane 0...")
usb4_results = usb4_dashboard.analyze_eye_diagram(lane='lane_0')

print(f"\n📊 USB4 Lane 0 Analysis Results:")
print(f"   Eye Height: {usb4_results['eye_height']:.4f} V")
print(f"   Eye Width: {usb4_results['eye_width']:.4f} UI")
print(f"   SNR: {usb4_results['snr']:.2f} dB")
print(f"   Q-Factor: {usb4_results['q_factor']:.2f}")
print(f"   Status: {'✅ PASS' if usb4_results['passed'] else '❌ FAIL'}")

In [ ]:
# Create static dashboard for USB4
print("📊 Creating static USB4 dashboard...")
usb4_dashboard.create_static_dashboard(lane='lane_0')

## 🎛️ Interactive Dashboard with Controls

Now let's create an interactive dashboard with real-time controls:

In [ ]:
# Create interactive dashboard for USB4
print("🎛️ Creating interactive USB4 dashboard...")
usb4_dashboard.create_interactive_dashboard()

## 🔄 Multi-Protocol Analysis

Let's analyze different protocols and compare their characteristics:

In [ ]:
# Create dashboards for all protocols
protocols_data = {
    'USB4': {'signals': usb4_signals, 'time': usb4_time, 'sample_rate': 40e9},
    'PCIe': {'signals': pcie_signals, 'time': pcie_time, 'sample_rate': 80e9},
    'Ethernet': {'signals': eth_signals, 'time': eth_time, 'sample_rate': 224e9}
}

dashboards = {}
analysis_results = {}

print("🔄 Creating dashboards for all protocols...")

for protocol, data in protocols_data.items():
    print(f"\n📊 Creating {protocol} dashboard...")
    
    # Create dashboard
    dashboard = create_dashboard(
        signal_data=data['signals'],
        sample_rate=data['sample_rate'],
        protocol=protocol
    )
    
    # Analyze first lane
    results = dashboard.analyze_eye_diagram(lane=0)
    
    dashboards[protocol] = dashboard
    analysis_results[protocol] = results
    
    print(f"   ✅ {protocol}: Eye Height={results['eye_height']:.4f}V, SNR={results['snr']:.2f}dB, Status={'PASS' if results['passed'] else 'FAIL'}")

print("\n🎯 Multi-protocol analysis complete!")

In [ ]:
# Create comparison table
import pandas as pd

comparison_data = []
for protocol, results in analysis_results.items():
    comparison_data.append({
        'Protocol': protocol,
        'Eye Height (V)': f"{results['eye_height']:.4f}",
        'Eye Width (UI)': f"{results['eye_width']:.4f}",
        'SNR (dB)': f"{results['snr']:.2f}",
        'Q-Factor': f"{results['q_factor']:.2f}",
        'Status': '✅ PASS' if results['passed'] else '❌ FAIL'
    })

comparison_df = pd.DataFrame(comparison_data)
print("📊 Protocol Comparison Summary:")
print(comparison_df.to_string(index=False))

## 📈 Advanced Waveform Analysis

Let's use the waveform analyzer for detailed signal analysis:

In [ ]:
# Create waveform analyzer for USB4
print("🔍 Creating waveform analyzer for USB4...")

waveform_analyzer = WaveformAnalyzer(sample_rate=40e9, protocol="USB4")

# Analyze both USB4 lanes
for lane_name, signal in usb4_signals.items():
    print(f"\n📊 Analyzing {lane_name}...")
    
    waveform_result = waveform_analyzer.analyze_waveform(
        voltage_data=signal,
        time_data=usb4_time,
        lane=lane_name
    )
    
    print(f"   Peak-to-Peak: {waveform_result.peak_to_peak:.4f} V")
    print(f"   SNR: {waveform_result.snr_db:.2f} dB")
    print(f"   THD: {waveform_result.thd_percent:.2f}%")
    print(f"   Status: {'✅ PASS' if waveform_result.passed else '❌ FAIL'}")
    
    if not waveform_result.passed:
        print(f"   Failures: {', '.join(waveform_result.failure_reasons)}")

print("\n📋 Generating summary report...")
summary_report = waveform_analyzer.get_summary_report()
print(summary_report)

In [ ]:
# Create waveform analysis plots
print("📊 Creating waveform analysis plots...")
waveform_analyzer.create_analysis_plot(lane='lane_0', plot_type='matplotlib')

## 🎨 Interactive Plotting Features

Let's explore the interactive plotting capabilities:

In [ ]:
# Create interactive plotter
from serdes_validation_framework.jupyter_dashboard.interactive_plots import PlotConfig

plot_config = PlotConfig(
    width=900,
    height=600,
    theme='plotly_white'
)

interactive_plotter = InteractivePlotter(config=plot_config)

print("🎨 Creating interactive eye diagram plot...")
interactive_plotter.create_interactive_eye_plot(
    signal_data=usb4_signals,
    time_data=usb4_time,
    sample_rate=40e9,
    protocol="USB4"
)

In [ ]:
# Create interactive spectrum analyzer
print("📊 Creating interactive spectrum analyzer...")
interactive_plotter.create_interactive_spectrum_plot(
    signal_data=usb4_signals,
    sample_rate=40e9,
    protocol="USB4"
)

In [ ]:
# Create multi-lane comparison
print("🔄 Creating multi-lane comparison...")
interactive_plotter.create_multi_lane_comparison(
    signal_data=usb4_signals,
    time_data=usb4_time,
    protocol="USB4"
)

In [ ]:
# Create measurement dashboard
print("📊 Creating measurement dashboard...")
interactive_plotter.create_measurement_dashboard(
    analysis_results=analysis_results,
    protocol="Multi-Protocol"
)

## 💾 Export and Save Results

Let's export our analysis results for further processing:

In [ ]:
# Export analysis results
print("💾 Exporting analysis results...")

# Export USB4 results
usb4_dashboard.export_results("usb4_eye_analysis.json")

# Export PCIe results
dashboards['PCIe'].export_results("pcie_eye_analysis.json")

# Export Ethernet results
dashboards['Ethernet'].export_results("ethernet_eye_analysis.json")

print("✅ All results exported successfully!")

In [ ]:
# Create a comprehensive summary
print("📋 Creating comprehensive analysis summary...")

summary = {
    'analysis_timestamp': pd.Timestamp.now().isoformat(),
    'protocols_analyzed': list(analysis_results.keys()),
    'total_lanes_analyzed': sum(len(data['signals']) for data in protocols_data.values()),
    'overall_pass_rate': sum(1 for r in analysis_results.values() if r['passed']) / len(analysis_results),
    'protocol_results': {}
}

for protocol, results in analysis_results.items():
    summary['protocol_results'][protocol] = {
        'eye_height': results['eye_height'],
        'snr_db': results['snr'],
        'passed': results['passed'],
        'sample_rate_ghz': protocols_data[protocol]['sample_rate'] / 1e9
    }

# Save summary as JSON
import json
with open('jupyter_dashboard_analysis_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print(f"📊 Analysis Summary:")
print(f"   Protocols Analyzed: {len(analysis_results)}")
print(f"   Overall Pass Rate: {summary['overall_pass_rate']:.1%}")
print(f"   Total Lanes: {summary['total_lanes_analyzed']}")
print(f"   Summary saved to: jupyter_dashboard_analysis_summary.json")

print("\n🎉 Jupyter Dashboard Analysis Complete!")

## 📚 Additional Resources

### 🔗 Useful Links
- [SerDes Validation Framework Documentation](../docs/)
- [Visualization Guide](../docs/guides/visualization.md)
- [API Reference](../docs/api/)

### 💡 Tips for Advanced Usage
1. **Custom Configurations**: Modify `DashboardConfig` for specific visualization needs
2. **Protocol Extensions**: Add support for new protocols by extending the analyzer classes
3. **Export Options**: Use various export formats (JSON, CSV, PNG) for different workflows
4. **Interactive Features**: Leverage Jupyter widgets for real-time parameter adjustment
5. **Integration**: Combine with stress testing and other framework features

### 🛠️ Troubleshooting
- **Missing Dependencies**: Run `check_dashboard_dependencies()` to verify installation
- **Performance Issues**: Reduce signal length or number of symbols for faster rendering
- **Widget Issues**: Restart kernel if interactive widgets stop responding
- **Export Problems**: Check file permissions and available disk space

---

**🎊 Congratulations! You've successfully used the SerDes Validation Framework Jupyter Dashboard to analyze eye diagrams, visualize waveforms, and generate professional reports for multiple protocols.**